# Railway Weather Correlation Analysis - Exploratory Data Analysis

## Research Questions

### Primary Research Question
**What is the correlation between meteorological conditions and railway delay patterns in the Lombardy region?**

### Secondary Research Questions
1. How do railway delays vary across different time scales (hourly, daily, seasonal)?
2. What are the spatial patterns of delays across different stations?
3. How do peak traffic periods interact with weather conditions?
4. What is the data quality and integration success rate?

---

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Database connection
import os
import sys
sys.path.append('src')
from database.db_manager import DatabaseManager

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("Libraries imported successfully!")

In [ ]:
# Initialize database connection and load data
db_manager = DatabaseManager()

# Load integrated dataset (main analysis dataset)
integrated_query = """
SELECT 
    train_id, timestamp, station_code, delay_minutes, temperature,
    wind_speed, precip_mm, weather_code, train_category, route,
    delay_status, destination, is_cancelled, hour_of_day, day_of_week,
    is_weekend, is_rush_hour, temp_category, is_raining, rain_intensity,
    wind_category, is_delayed, delay_category
FROM train_weather_integrated
ORDER BY timestamp
"""

df = db_manager.execute_query(integrated_query)
df['timestamp'] = pd.to_datetime(df['timestamp'])

print(f"Loaded {len(df):,} integrated records")
print(f"Date range: {df['timestamp'].min()} to {df['timestamp'].max()}")
print(f"Columns: {list(df.columns)}")

## 1. Data Overview and Quality Assessment

In [ ]:
# Basic dataset information
print("Dataset Overview:")
print(f"Shape: {df.shape}")
print(f"Unique stations: {df['station_code'].nunique()}")
print(f"Unique trains: {df['train_id'].nunique()}")
print(f"Date range: {(df['timestamp'].max() - df['timestamp'].min()).days} days")

# Display basic statistics
df.info()

In [ ]:
# Missing values analysis
missing_data = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df) * 100).round(2)
}).sort_values('Missing_Percentage', ascending=False)

print("Missing Values Analysis:")
print(missing_data)

# Plot missing values
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Bar chart of missing percentages
missing_pct = missing_data[missing_data['Missing_Percentage'] > 0]
if not missing_pct.empty:
    ax1.barh(missing_pct['Column'], missing_pct['Missing_Percentage'], color='coral')
    ax1.set_xlabel('Missing Percentage')
    ax1.set_title('Missing Values by Column')

# Heatmap of missing values pattern
if df.isnull().any().any():
    sns.heatmap(df.isnull().iloc[:1000], ax=ax2, cbar=True, yticklabels=False, cmap='viridis')
    ax2.set_title('Missing Values Pattern (First 1000 records)')

plt.tight_layout()
plt.show()

## 2. Delay Distribution Analysis

In [ ]:
# Delay statistics
if 'delay_minutes' in df.columns:
    delay_data = df['delay_minutes'].dropna()
    
    print("Delay Statistics:")
    print(f"Mean delay: {delay_data.mean():.2f} minutes")
    print(f"Median delay: {delay_data.median():.2f} minutes")
    print(f"Standard deviation: {delay_data.std():.2f} minutes")
    print(f"Max delay: {delay_data.max():.2f} minutes")
    print(f"Trains with delays >5 min: {(delay_data > 5).sum():,} ({(delay_data > 5).mean()*100:.1f}%)")
    print(f"Trains with delays >15 min: {(delay_data > 15).sum():,} ({(delay_data > 15).mean()*100:.1f}%)")
    
    # Remove extreme outliers for visualization
    q99 = delay_data.quantile(0.99)
    delay_clean = delay_data[delay_data <= q99]
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Histogram
    axes[0,0].hist(delay_clean, bins=50, alpha=0.7, color='skyblue', edgecolor='black')
    axes[0,0].axvline(delay_clean.mean(), color='red', linestyle='--', label=f'Mean: {delay_clean.mean():.1f} min')
    axes[0,0].axvline(delay_clean.median(), color='orange', linestyle='--', label=f'Median: {delay_clean.median():.1f} min')
    axes[0,0].set_title('Distribution of Train Delays')
    axes[0,0].set_xlabel('Delay (minutes)')
    axes[0,0].set_ylabel('Frequency')
    axes[0,0].legend()
    
    # Box plot by delay status
    if 'delay_status' in df.columns:
        delay_status_data = df[df['delay_minutes'].notna() & df['delay_status'].notna()]
        if not delay_status_data.empty:
            sns.boxplot(data=delay_status_data, x='delay_status', y='delay_minutes', ax=axes[0,1])
            axes[0,1].set_title('Delays by Status Category')
            axes[0,1].tick_params(axis='x', rotation=45)
    
    # Delays by hour
    if 'hour_of_day' in df.columns:
        hourly_delays = df.groupby('hour_of_day')['delay_minutes'].mean()
        axes[1,0].bar(hourly_delays.index, hourly_delays.values, alpha=0.7, color='lightgreen')
        axes[1,0].set_title('Average Delays by Hour of Day')
        axes[1,0].set_xlabel('Hour of Day')
        axes[1,0].set_ylabel('Average Delay (minutes)')
    
    # Delays by day of week
    if 'day_of_week' in df.columns:
        daily_delays = df.groupby('day_of_week')['delay_minutes'].mean()
        day_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
        axes[1,1].bar(range(len(daily_delays)), daily_delays.values, alpha=0.7, color='orange')
        axes[1,1].set_title('Average Delays by Day of Week')
        axes[1,1].set_xlabel('Day of Week')
        axes[1,1].set_ylabel('Average Delay (minutes)')
        axes[1,1].set_xticks(range(len(day_names)))
        axes[1,1].set_xticklabels(day_names)
    
    plt.tight_layout()
    plt.show()

## 3. Weather-Delay Correlation Analysis

In [ ]:
# Weather-delay correlation analysis
complete_data = df.dropna(subset=['delay_minutes', 'temperature'])

if not complete_data.empty:
    print(f"Complete weather-delay records: {len(complete_data):,} ({len(complete_data)/len(df)*100:.1f}% of total)")
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Temperature vs Delay
    axes[0,0].scatter(complete_data['temperature'], complete_data['delay_minutes'], alpha=0.5, s=20)
    z = np.polyfit(complete_data['temperature'], complete_data['delay_minutes'], 1)
    p = np.poly1d(z)
    axes[0,0].plot(complete_data['temperature'], p(complete_data['temperature']), "r--", alpha=0.8)
    axes[0,0].set_xlabel('Temperature (°C)')
    axes[0,0].set_ylabel('Delay (minutes)')
    axes[0,0].set_title('Temperature vs Train Delays')
    
    # Calculate correlation
    temp_corr = complete_data['temperature'].corr(complete_data['delay_minutes'])
    axes[0,0].text(0.05, 0.95, f'Correlation: {temp_corr:.3f}', transform=axes[0,0].transAxes, 
                   bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # Precipitation vs Delay
    if 'precip_mm' in complete_data.columns:
        precip_data = complete_data[complete_data['precip_mm'].notna()]
        if not precip_data.empty:
            axes[0,1].scatter(precip_data['precip_mm'], precip_data['delay_minutes'], alpha=0.5, s=20, color='blue')
            axes[0,1].set_xlabel('Precipitation (mm)')
            axes[0,1].set_ylabel('Delay (minutes)')
            axes[0,1].set_title('Precipitation vs Train Delays')
            
            precip_corr = precip_data['precip_mm'].corr(precip_data['delay_minutes'])
            axes[0,1].text(0.05, 0.95, f'Correlation: {precip_corr:.3f}', transform=axes[0,1].transAxes,
                          bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # Wind speed vs Delay
    if 'wind_speed' in complete_data.columns:
        wind_data = complete_data[complete_data['wind_speed'].notna()]
        if not wind_data.empty:
            axes[1,0].scatter(wind_data['wind_speed'], wind_data['delay_minutes'], alpha=0.5, s=20, color='green')
            axes[1,0].set_xlabel('Wind Speed (km/h)')
            axes[1,0].set_ylabel('Delay (minutes)')
            axes[1,0].set_title('Wind Speed vs Train Delays')
            
            wind_corr = wind_data['wind_speed'].corr(wind_data['delay_minutes'])
            axes[1,0].text(0.05, 0.95, f'Correlation: {wind_corr:.3f}', transform=axes[1,0].transAxes,
                          bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # Rain vs No Rain comparison
    if 'is_raining' in complete_data.columns:
        rain_comparison = complete_data.groupby('is_raining')['delay_minutes'].agg(['mean', 'count', 'std'])
        rain_labels = ['No Rain', 'Rain']
        
        bars = axes[1,1].bar(range(len(rain_comparison)), rain_comparison['mean'], 
                            yerr=rain_comparison['std'], capsize=5,
                            color=['lightblue', 'darkblue'], alpha=0.7)
        axes[1,1].set_title('Average Delays: Rain vs No Rain')
        axes[1,1].set_ylabel('Average Delay (minutes)')
        axes[1,1].set_xticks(range(len(rain_labels)))
        axes[1,1].set_xticklabels(rain_labels)
        
        # Add value labels on bars
        for i, (bar, mean_val, count) in enumerate(zip(bars, rain_comparison['mean'], rain_comparison['count'])):
            axes[1,1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
                          f'{mean_val:.1f}\n(n={count})', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
else:
    print("No complete weather-delay data available for correlation analysis")

## 4. Temporal Pattern Analysis

In [ ]:
# Temporal patterns analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Daily trends over time
if 'timestamp' in df.columns and 'delay_minutes' in df.columns:
    df['date'] = df['timestamp'].dt.date
    daily_stats = df.groupby('date').agg({
        'delay_minutes': ['mean', 'count'],
        'temperature': 'mean',
        'precip_mm': 'sum'
    }).reset_index()
    
    daily_stats.columns = ['date', 'avg_delay', 'train_count', 'avg_temp', 'total_precip']
    daily_stats['date'] = pd.to_datetime(daily_stats['date'])
    
    axes[0,0].plot(daily_stats['date'], daily_stats['avg_delay'], marker='o', markersize=3, linewidth=1)
    axes[0,0].set_title('Daily Average Delays Over Time')
    axes[0,0].set_xlabel('Date')
    axes[0,0].set_ylabel('Average Delay (minutes)')
    axes[0,0].tick_params(axis='x', rotation=45)

# Rush hour vs non-rush hour
if 'is_rush_hour' in df.columns and 'delay_minutes' in df.columns:
    rush_stats = df.groupby('is_rush_hour')['delay_minutes'].agg(['mean', 'count', 'std'])
    rush_labels = ['Non-Rush Hour', 'Rush Hour']
    
    bars = axes[0,1].bar(range(len(rush_stats)), rush_stats['mean'], 
                        yerr=rush_stats['std'], capsize=5,
                        color=['lightcoral', 'darkred'], alpha=0.7)
    axes[0,1].set_title('Average Delays: Rush Hour vs Non-Rush Hour')
    axes[0,1].set_ylabel('Average Delay (minutes)')
    axes[0,1].set_xticks(range(len(rush_labels)))
    axes[0,1].set_xticklabels(rush_labels)
    
    # Add statistical significance test
    from scipy import stats
    rush_delays = df[df['is_rush_hour'] == True]['delay_minutes'].dropna()
    non_rush_delays = df[df['is_rush_hour'] == False]['delay_minutes'].dropna()
    if len(rush_delays) > 0 and len(non_rush_delays) > 0:
        t_stat, p_value = stats.ttest_ind(rush_delays, non_rush_delays)
        axes[0,1].text(0.5, 0.95, f'p-value: {p_value:.4f}', transform=axes[0,1].transAxes,
                      ha='center', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

# Weekend vs weekday
if 'is_weekend' in df.columns and 'delay_minutes' in df.columns:
    weekend_stats = df.groupby('is_weekend')['delay_minutes'].agg(['mean', 'count', 'std'])
    weekend_labels = ['Weekday', 'Weekend']
    
    axes[1,0].bar(range(len(weekend_stats)), weekend_stats['mean'], 
                 yerr=weekend_stats['std'], capsize=5,
                 color=['lightgreen', 'darkgreen'], alpha=0.7)
    axes[1,0].set_title('Average Delays: Weekday vs Weekend')
    axes[1,0].set_ylabel('Average Delay (minutes)')
    axes[1,0].set_xticks(range(len(weekend_labels)))
    axes[1,0].set_xticklabels(weekend_labels)

# Hourly heatmap (day of week vs hour of day)
if 'hour_of_day' in df.columns and 'day_of_week' in df.columns:
    hourly_heatmap = df.pivot_table(
        values='delay_minutes', 
        index='day_of_week', 
        columns='hour_of_day', 
        aggfunc='mean'
    )
    
    if not hourly_heatmap.empty:
        sns.heatmap(hourly_heatmap, ax=axes[1,1], cmap='YlOrRd', 
                   cbar_kws={'label': 'Average Delay (min)'}, annot=False)
        axes[1,1].set_title('Average Delays Heatmap (Day vs Hour)')
        axes[1,1].set_xlabel('Hour of Day')
        axes[1,1].set_ylabel('Day of Week (0=Monday)')

plt.tight_layout()
plt.show()

## 5. Station and Geographic Analysis

In [ ]:
# Station analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Top stations by average delay
if 'delay_minutes' in df.columns:
    station_stats = df.groupby('station_code').agg({
        'delay_minutes': ['mean', 'count', 'std'],
        'train_id': 'nunique'
    }).reset_index()
    
    station_stats.columns = ['station_code', 'avg_delay', 'total_records', 'delay_std', 'unique_trains']
    
    # Filter for statistical significance (at least 50 records)
    significant_stations = station_stats[station_stats['total_records'] >= 50]
    top_delay_stations = significant_stations.nlargest(15, 'avg_delay')
    
    axes[0,0].barh(range(len(top_delay_stations)), top_delay_stations['avg_delay'])
    axes[0,0].set_yticks(range(len(top_delay_stations)))
    axes[0,0].set_yticklabels(top_delay_stations['station_code'])
    axes[0,0].set_title('Top 15 Stations by Average Delay\n(≥50 records)')
    axes[0,0].set_xlabel('Average Delay (minutes)')

# Top stations by volume
top_volume_stations = station_stats.nlargest(15, 'total_records')
axes[0,1].bar(range(len(top_volume_stations)), top_volume_stations['total_records'])
axes[0,1].set_xticks(range(len(top_volume_stations)))
axes[0,1].set_xticklabels(top_volume_stations['station_code'], rotation=45)
axes[0,1].set_title('Top 15 Stations by Train Volume')
axes[0,1].set_ylabel('Number of Records')

# Train category analysis
if 'train_category' in df.columns:
    category_stats = df.groupby('train_category').agg({
        'delay_minutes': ['mean', 'count', 'std']
    }).reset_index()
    category_stats.columns = ['train_category', 'avg_delay', 'count', 'delay_std']
    
    # Filter for categories with sufficient data
    significant_categories = category_stats[category_stats['count'] >= 20]
    
    if not significant_categories.empty:
        axes[1,0].bar(range(len(significant_categories)), significant_categories['avg_delay'],
                     yerr=significant_categories['delay_std'], capsize=5, alpha=0.7)
        axes[1,0].set_xticks(range(len(significant_categories)))
        axes[1,0].set_xticklabels(significant_categories['train_category'], rotation=45)
        axes[1,0].set_title('Average Delays by Train Category\n(≥20 records)')
        axes[1,0].set_ylabel('Average Delay (minutes)')

# Delay status distribution
if 'delay_status' in df.columns:
    status_counts = df['delay_status'].value_counts()
    colors = plt.cm.Set3(np.linspace(0, 1, len(status_counts)))
    wedges, texts, autotexts = axes[1,1].pie(status_counts.values, labels=status_counts.index, 
                                            autopct='%1.1f%%', colors=colors)
    axes[1,1].set_title('Distribution of Delay Status')

plt.tight_layout()
plt.show()

# Print station statistics
print("\nStation Statistics Summary:")
print(f"Total stations: {len(station_stats)}")
print(f"Stations with ≥50 records: {len(significant_stations)}")
print(f"Highest average delay: {top_delay_stations.iloc[0]['avg_delay']:.2f} min at {top_delay_stations.iloc[0]['station_code']}")
print(f"Highest volume station: {top_volume_stations.iloc[0]['station_code']} with {top_volume_stations.iloc[0]['total_records']} records")

## 6. Correlation Matrix and Statistical Analysis

In [ ]:
# Correlation matrix for key variables
numeric_cols = ['delay_minutes', 'temperature', 'wind_speed', 'precip_mm', 'hour_of_day', 'day_of_week']
available_cols = [col for col in numeric_cols if col in df.columns]

if len(available_cols) >= 2:
    correlation_data = df[available_cols].corr()
    
    plt.figure(figsize=(10, 8))
    mask = np.triu(np.ones_like(correlation_data, dtype=bool))  # Show only lower triangle
    sns.heatmap(correlation_data, mask=mask, annot=True, cmap='RdBu_r', center=0, 
               square=True, fmt='.3f', cbar_kws={'label': 'Correlation Coefficient'})
    plt.title('Correlation Matrix: Weather and Delay Variables')
    plt.tight_layout()
    plt.show()
    
    # Print key correlations with delay
    if 'delay_minutes' in correlation_data.columns:
        print("\nCorrelations with Delay Minutes:")
        delay_corr = correlation_data['delay_minutes'].drop('delay_minutes').sort_values(key=abs, ascending=False)
        for var, corr in delay_corr.items():
            significance = "***" if abs(corr) > 0.1 else "**" if abs(corr) > 0.05 else "*" if abs(corr) > 0.01 else ""
            print(f"  {var}: {corr:.4f} {significance}")
        
        print("\nSignificance levels: *** |r| > 0.1, ** |r| > 0.05, * |r| > 0.01")
else:
    print("Insufficient numeric data for correlation analysis")

## 7. Advanced Weather Impact Analysis

In [ ]:
# Advanced weather impact analysis
if 'temperature' in df.columns and 'delay_minutes' in df.columns:
    # Create temperature categories
    df['temp_range'] = pd.cut(df['temperature'], 
                             bins=[-np.inf, 0, 10, 20, 30, np.inf], 
                             labels=['<0°C', '0-10°C', '10-20°C', '20-30°C', '>30°C'])
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Delays by temperature range
    temp_delays = df.groupby('temp_range')['delay_minutes'].agg(['mean', 'count', 'std']).reset_index()
    temp_delays = temp_delays[temp_delays['count'] >= 10]  # Filter for significance
    
    if not temp_delays.empty:
        axes[0,0].bar(range(len(temp_delays)), temp_delays['mean'], 
                     yerr=temp_delays['std'], capsize=5, alpha=0.7)
        axes[0,0].set_xticks(range(len(temp_delays)))
        axes[0,0].set_xticklabels(temp_delays['temp_range'])
        axes[0,0].set_title('Average Delays by Temperature Range')
        axes[0,0].set_ylabel('Average Delay (minutes)')
    
    # Precipitation impact
    if 'precip_mm' in df.columns:
        df['rain_category'] = pd.cut(df['precip_mm'], 
                                   bins=[-0.1, 0, 1, 5, 20, np.inf], 
                                   labels=['No Rain', 'Light', 'Moderate', 'Heavy', 'Very Heavy'])
        
        rain_delays = df.groupby('rain_category')['delay_minutes'].agg(['mean', 'count', 'std']).reset_index()
        rain_delays = rain_delays[rain_delays['count'] >= 5]
        
        if not rain_delays.empty:
            axes[0,1].bar(range(len(rain_delays)), rain_delays['mean'], 
                         yerr=rain_delays['std'], capsize=5, alpha=0.7, color='blue')
            axes[0,1].set_xticks(range(len(rain_delays)))
            axes[0,1].set_xticklabels(rain_delays['rain_category'], rotation=45)
            axes[0,1].set_title('Average Delays by Precipitation Intensity')
            axes[0,1].set_ylabel('Average Delay (minutes)')
    
    # Combined weather conditions
    if 'is_raining' in df.columns and 'temp_range' in df.columns:
        combined_weather = df.groupby(['temp_range', 'is_raining'])['delay_minutes'].mean().unstack(fill_value=0)
        
        if not combined_weather.empty:
            combined_weather.plot(kind='bar', ax=axes[1,0], alpha=0.7)
            axes[1,0].set_title('Average Delays by Temperature and Rain')
            axes[1,0].set_ylabel('Average Delay (minutes)')
            axes[1,0].set_xlabel('Temperature Range')
            axes[1,0].legend(['No Rain', 'Rain'])
            axes[1,0].tick_params(axis='x', rotation=45)
    
    # Weather vs Rush Hour interaction
    if 'is_raining' in df.columns and 'is_rush_hour' in df.columns:
        weather_rush = df.groupby(['is_raining', 'is_rush_hour'])['delay_minutes'].mean().unstack(fill_value=0)
        
        if not weather_rush.empty:
            weather_rush.plot(kind='bar', ax=axes[1,1], alpha=0.7)
            axes[1,1].set_title('Weather-Rush Hour Interaction')
            axes[1,1].set_ylabel('Average Delay (minutes)')
            axes[1,1].set_xlabel('Rain Condition')
            axes[1,1].legend(['Non-Rush', 'Rush Hour'])
            axes[1,1].set_xticklabels(['No Rain', 'Rain'], rotation=0)
    
    plt.tight_layout()
    plt.show()

## 8. Data Quality and Integration Success Analysis

In [ ]:
# Data quality analysis
print("Data Quality Assessment:")
print("=" * 40)

# Overall completeness
total_cells = len(df) * len(df.columns)
missing_cells = df.isnull().sum().sum()
completeness = (1 - missing_cells / total_cells) * 100

print(f"Overall Data Completeness: {completeness:.2f}%")

# Weather integration success rate
weather_match_rate = (df['temperature'].notna()).mean() * 100
print(f"Weather Integration Success Rate: {weather_match_rate:.2f}%")

# Key field completeness
key_fields = ['delay_minutes', 'temperature', 'wind_speed', 'precip_mm']
print("\nKey Field Completeness:")
for field in key_fields:
    if field in df.columns:
        completeness = (df[field].notna()).mean() * 100
        print(f"  {field}: {completeness:.2f}%")

# Temporal coverage
if 'timestamp' in df.columns:
    date_range = df['timestamp'].max() - df['timestamp'].min()
    unique_dates = df['timestamp'].dt.date.nunique()
    print(f"\nTemporal Coverage:")
    print(f"  Total date range: {date_range.days} days")
    print(f"  Days with data: {unique_dates}")
    print(f"  Coverage rate: {unique_dates/date_range.days*100:.2f}%")

# Data quality visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Daily data volume
if 'timestamp' in df.columns:
    daily_volume = df.groupby(df['timestamp'].dt.date).size()
    axes[0,0].plot(daily_volume.index, daily_volume.values, marker='o', markersize=2)
    axes[0,0].set_title('Daily Data Volume')
    axes[0,0].set_xlabel('Date')
    axes[0,0].set_ylabel('Number of Records')
    axes[0,0].tick_params(axis='x', rotation=45)

# Completeness by field
completeness_by_field = (1 - df.isnull().mean()) * 100
completeness_by_field = completeness_by_field.sort_values()

axes[0,1].barh(range(len(completeness_by_field)), completeness_by_field.values)
axes[0,1].set_yticks(range(len(completeness_by_field)))
axes[0,1].set_yticklabels(completeness_by_field.index)
axes[0,1].set_title('Data Completeness by Field')
axes[0,1].set_xlabel('Completeness (%)')
axes[0,1].set_xlim(0, 100)

# Missing data pattern over time
if 'timestamp' in df.columns:
    df['date'] = df['timestamp'].dt.date
    daily_missing = df.groupby('date').apply(lambda x: x.isnull().mean() * 100)
    
    if 'temperature' in daily_missing.columns:
        axes[1,0].plot(daily_missing.index, daily_missing['temperature'], 
                      marker='o', markersize=2, label='Temperature')
    if 'delay_minutes' in daily_missing.columns:
        axes[1,0].plot(daily_missing.index, daily_missing['delay_minutes'], 
                      marker='s', markersize=2, label='Delay Minutes')
    
    axes[1,0].set_title('Missing Data Trends Over Time')
    axes[1,0].set_xlabel('Date')
    axes[1,0].set_ylabel('Missing Percentage')
    axes[1,0].legend()
    axes[1,0].tick_params(axis='x', rotation=45)

# Data quality score by table
quality_scores = {
    'Integrated Data': completeness,
    'Weather Match': weather_match_rate,
    'Delay Data': (df['delay_minutes'].notna()).mean() * 100 if 'delay_minutes' in df.columns else 0
}

axes[1,1].bar(quality_scores.keys(), quality_scores.values(), 
             color=['skyblue', 'lightgreen', 'coral'], alpha=0.7)
axes[1,1].set_title('Data Quality Scores')
axes[1,1].set_ylabel('Quality Score (%)')
axes[1,1].set_ylim(0, 100)

# Add value labels on bars
for i, (key, value) in enumerate(quality_scores.items()):
    axes[1,1].text(i, value + 1, f'{value:.1f}%', ha='center', va='bottom')

plt.tight_layout()
plt.show()

## 9. Summary and Key Findings

In [ ]:
# Generate comprehensive summary
print("\n" + "="*80)
print("RAILWAY WEATHER CORRELATION ANALYSIS - KEY FINDINGS")
print("="*80)

# Dataset summary
print(f"\n📊 DATASET OVERVIEW:")
print(f"   • Total Records: {len(df):,}")
print(f"   • Date Range: {df['timestamp'].min().strftime('%Y-%m-%d')} to {df['timestamp'].max().strftime('%Y-%m-%d')}")
print(f"   • Unique Stations: {df['station_code'].nunique()}")
print(f"   • Unique Trains: {df['train_id'].nunique()}")
print(f"   • Data Completeness: {completeness:.1f}%")
print(f"   • Weather Integration Rate: {weather_match_rate:.1f}%")

# Delay analysis summary
if 'delay_minutes' in df.columns:
    delay_data = df['delay_minutes'].dropna()
    print(f"\n🚂 DELAY PATTERNS:")
    print(f"   • Average Delay: {delay_data.mean():.2f} minutes")
    print(f"   • Median Delay: {delay_data.median():.2f} minutes")
    print(f"   • Trains with Significant Delays (>5 min): {(delay_data > 5).mean()*100:.1f}%")
    print(f"   • Trains with Major Delays (>15 min): {(delay_data > 15).mean()*100:.1f}%")

# Weather correlation findings
if not complete_data.empty:
    temp_corr = complete_data['temperature'].corr(complete_data['delay_minutes'])
    print(f"\n🌤️ WEATHER CORRELATIONS:")
    print(f"   • Temperature-Delay Correlation: {temp_corr:.4f}")
    
    if 'precip_mm' in complete_data.columns:
        precip_data = complete_data[complete_data['precip_mm'].notna()]
        if not precip_data.empty:
            precip_corr = precip_data['precip_mm'].corr(precip_data['delay_minutes'])
            print(f"   • Precipitation-Delay Correlation: {precip_corr:.4f}")
    
    if 'is_raining' in complete_data.columns:
        rain_delays = complete_data[complete_data['is_raining'] == True]['delay_minutes'].mean()
        no_rain_delays = complete_data[complete_data['is_raining'] == False]['delay_minutes'].mean()
        rain_impact = ((rain_delays - no_rain_delays) / no_rain_delays) * 100
        print(f"   • Rain Impact: {rain_impact:+.1f}% change in average delays")

# Temporal patterns
if 'is_rush_hour' in df.columns and 'delay_minutes' in df.columns:
    rush_delays = df[df['is_rush_hour'] == True]['delay_minutes'].mean()
    non_rush_delays = df[df['is_rush_hour'] == False]['delay_minutes'].mean()
    rush_impact = ((rush_delays - non_rush_delays) / non_rush_delays) * 100
    print(f"\n⏰ TEMPORAL PATTERNS:")
    print(f"   • Rush Hour Impact: {rush_impact:+.1f}% change in average delays")
    
    if 'is_weekend' in df.columns:
        weekend_delays = df[df['is_weekend'] == True]['delay_minutes'].mean()
        weekday_delays = df[df['is_weekend'] == False]['delay_minutes'].mean()
        weekend_impact = ((weekend_delays - weekday_delays) / weekday_delays) * 100
        print(f"   • Weekend Impact: {weekend_impact:+.1f}% change in average delays")

# Station insights
if 'station_code' in df.columns and 'delay_minutes' in df.columns:
    station_stats = df.groupby('station_code')['delay_minutes'].agg(['mean', 'count'])
    significant_stations = station_stats[station_stats['count'] >= 50]
    print(f"\n🚉 STATION ANALYSIS:")
    print(f"   • Stations with Sufficient Data (≥50 records): {len(significant_stations)}")
    if not significant_stations.empty:
        worst_station = significant_stations['mean'].idxmax()
        best_station = significant_stations['mean'].idxmin()
        print(f"   • Highest Average Delays: {worst_station} ({significant_stations.loc[worst_station, 'mean']:.2f} min)")
        print(f"   • Lowest Average Delays: {best_station} ({significant_stations.loc[best_station, 'mean']:.2f} min)")

# Research questions answered
print(f"\n✅ RESEARCH QUESTIONS ADDRESSED:")
print(f"   1. ✓ Primary: Weather-delay correlations quantified")
print(f"   2. ✓ Temporal: Rush hour and seasonal patterns analyzed")
print(f"   3. ✓ Spatial: Station-level delay patterns identified")
print(f"   4. ✓ Operational: Train categories and traffic impacts assessed")
print(f"   5. ✓ Data Quality: Integration success and completeness measured")

print(f"\n📈 ANALYSIS COMPLETE - {len([f for f in os.listdir('.') if f.endswith('.png')])} visualizations generated")
print("="*80)